In [1]:
import json
import pickle
import numpy as np
import pandas as pd
#from sklearn.externals import joblib
from azureml.core.model import Model

In [2]:
import joblib

In [8]:
#Let's connect to our workspace:
from azureml.core import Workspace
ws_new = Workspace.from_config(path=".azureml/fc_config.json")
ws_new

Workspace.create(name='fraud_detect', subscription_id='53dc328a-1da3-4f32-a012-85cebcd27290', resource_group='Arpita')

In [9]:
#sample = json.dumps({"data": [1,7234,2,3,0,1,1,2,1,2]})
#3160241992,500,26000,N,0,Y,Y,800,677.2,6,Y
data = pd.DataFrame(data=[{
              "input_name_1": 3160241992,        
              "input_name_2": 500,    
              "input_name_3": 26000,
              "input_name_4": 0,        
              "input_name_5": 0,    
              "input_name_6": 1,
              "input_name_7": 1,        
              "input_name_8": 800,    
              "input_name_9": 677.2,
              "input_name_10":6 
                }])
#print(sample)
#print(type(sample))
#print(json.loads(sample)['data'])
#my_array = np.array(json.loads(sample)['data'])
#print(my_array.shape)
#my2d_array = my_array.reshape(-1,1)
#print(my2d_array)
#print(my2d_array.shape)
##test_samples = bytes(test_samples, encoding='utf8')

#print(Model.list(ws))
model_path = Model.get_model_path('frauddetection')
print(model_path)

local_model = joblib.load(model_path)
print("local_model: ", local_model.predict(data))

def run(data):
    #data = np.array(json.loads(data)['data'])
    result = local_model.predict(data)
    
    return result.tolist()
    
print("local_model_through_run_function: ", run(data))  


azureml-models/frauddetection/1/model_fd1.pkl
local_model:  [0]
local_model_through_run_function:  [0]


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [10]:
local_model = joblib.load(model_path)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [13]:
%%writefile score.py
import json
import pickle
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from azureml.core.model import Model



def init():
    global model
    model_path = Model.get_model_path(model_name = 'frauddetection')
    model = joblib.load(model_path)
    
    
    
def run(data):
    #data = np.array(json.loads(data)['data'])
    result = model.predict(data)
    
    return result.tolist()

Overwriting score.py


In [28]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.set_python_version("3.6.6")
dependencies.add_pip_package("scikit-learn==0.19.1")
dependencies.add_pip_package("numpy==1.17.3")
dependencies.add_pip_package("pandas==0.19.2")
dependencies.add_pip_package("joblib==0.14.1")
#dependencies.add_pip_package("matplotlib==3.0.0")

dependencies.save_to_file(".","myenv.yml")

'myenv.yml'

In [29]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage

In [30]:
# Define the configuration of compute: ACI with 1 cpu core and 1 gb of memory.
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
aci_config



In [31]:
# Build a container image
image_config = ContainerImage.image_configuration(execution_script = "score.py", 
                                   runtime = "python", 
                                   conda_file = "myenv.yml")
image_config


In [32]:
# from azureml.core import Workspace
# ws_new = Workspace.from_config()
# ws_new

In [33]:
model_path = Model.get_model_path('frauddetection')
model = Model(ws_new,'frauddetection')

In [ ]:
# Deploy the web service as an image containing the registered model.

service = Webservice.deploy_from_model(name = "frauddetection",
                                        deployment_config = aci_config,
                                        models = [model],
                                        image_config = image_config,
                                        workspace = ws_new)

# The service deployment can take several minutes: wait for completion.
service.wait_for_deployment(show_output = True)

Creating image
Running.............................................
Succeeded
Image creation operation finished for image frauddetection:1, operation "Succeeded"
Running...............................................................................................................................................................................................................................................

In [27]:
#Webservice.deploy_from_image(workspace=ws_new,name = "frauddetection1", image = "fraud-detection", deployment_config=aci_config)
# The service deployment can take several minutes: wait for completion.
#service.wait_for_deployment(show_output = True)

In [ ]:
service.get_logs()

In [ ]:
service.delete()

In [ ]:
sample = json.dumps({"data": [3160041896,185.5,4823,1,5,0,0,0,0,0]})
print(sample)
result = service.run(sample)
#json.loads(sample)['data']
print("return_from_aci_deployment: ", result)
#my_model = joblib.load(model_path)
#data = np.array([1,3,5,7])
#data
#data.size